Install the dependencies:

In [1]:
!pip install torch
!pip install torchvision
!pip install tensorflow
!pip install onnx
!pip install -U tf2onnx

Load PyTorch model:

In [2]:
import torch
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

torch_model = mobilenet_v3_small(
  weights=MobileNet_V3_Small_Weights.DEFAULT)

Define input and output names for better readability:

In [3]:
torch.onnx.export(
    model=torch_model,
    args=torch.randn(1, 3, 224, 224),
    f='torch_model.onnx',
    export_params=True,
    verbose=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Verify the exported model with ONNX:

In [4]:
import onnx

torch_model_onnx = onnx.load('torch_model.onnx')
onnx.checker.check_model(torch_model_onnx)

print(onnx.helper.printable_graph(torch_model_onnx.graph))

graph torch_jit (
  %input.1[FLOAT, 1x3x224x224]
) initializers (
  %features.1.block.1.fc1.weight[FLOAT, 8x16x1x1]
  %features.1.block.1.fc1.bias[FLOAT, 8]
  %features.1.block.1.fc2.weight[FLOAT, 16x8x1x1]
  %features.1.block.1.fc2.bias[FLOAT, 16]
  %features.4.block.2.fc1.weight[FLOAT, 24x96x1x1]
  %features.4.block.2.fc1.bias[FLOAT, 24]
  %features.4.block.2.fc2.weight[FLOAT, 96x24x1x1]
  %features.4.block.2.fc2.bias[FLOAT, 96]
  %features.5.block.2.fc1.weight[FLOAT, 64x240x1x1]
  %features.5.block.2.fc1.bias[FLOAT, 64]
  %features.5.block.2.fc2.weight[FLOAT, 240x64x1x1]
  %features.5.block.2.fc2.bias[FLOAT, 240]
  %features.6.block.2.fc1.weight[FLOAT, 64x240x1x1]
  %features.6.block.2.fc1.bias[FLOAT, 64]
  %features.6.block.2.fc2.weight[FLOAT, 240x64x1x1]
  %features.6.block.2.fc2.bias[FLOAT, 240]
  %features.7.block.2.fc1.weight[FLOAT, 32x120x1x1]
  %features.7.block.2.fc1.bias[FLOAT, 32]
  %features.7.block.2.fc2.weight[FLOAT, 120x32x1x1]
  %features.7.block.2.fc2.bias[FLOAT, 120

Let's continue with TF. First, load the model:

In [5]:
import tensorflow as tf

tf_model = tf.keras.applications.MobileNetV3Small(
  weights='imagenet',
  input_shape=(224, 224, 3),
)

Serialize the model:

In [10]:
import tf2onnx

tf_model_onnx, _ = tf2onnx.convert.from_keras(
  model=tf_model,
  input_signature=[tf.TensorSpec([1, 224, 224, 3])])
onnx.save(tf_model_onnx, 'tf_model.onnx')

Verify the model with ONNX:

In [7]:
tf_model_onnx = onnx.load('tf_model.onnx')
onnx.checker.check_model(tf_model_onnx)

print(onnx.helper.printable_graph(tf_model_onnx.graph))

graph tf2onnx (
  %args_0[FLOAT, unk__908x224x224x3]
) initializers (
  %pad_const__54[INT64, 8]
  %pad_const__317[INT64, 8]
  %const_fold_opt__902[FLOAT, 1x1x1x1]
  %const_fold_opt__842[INT64, 2]
  %MobilenetV3small/tf.math.multiply_9/Mul/y:0[FLOAT, scalar]
  %MobilenetV3small/rescaling/Cast_1/x:0[FLOAT, scalar]
  %MobilenetV3small/rescaling/Cast/x:0[FLOAT, scalar]
  %MobilenetV3small/re_lu_18/Relu6_max__257[FLOAT, scalar]
  %MobilenetV3small/re_lu_17/Relu6_min__246[FLOAT, scalar]
  %MobilenetV3small/expanded_conv_9/squeeze_excite/Conv_1/Conv2D/ReadVariableOp:0[FLOAT, 576x144x1x1]
  %MobilenetV3small/expanded_conv_9/squeeze_excite/Conv_1/BiasAdd/ReadVariableOp:0[FLOAT, 576]
  %MobilenetV3small/expanded_conv_9/squeeze_excite/Conv/Conv2D/ReadVariableOp:0[FLOAT, 144x576x1x1]
  %MobilenetV3small/expanded_conv_9/squeeze_excite/Conv/BiasAdd/ReadVariableOp:0[FLOAT, 144]
  %MobilenetV3small/expanded_conv_9/project/Conv2D_weights_fused_bn[FLOAT, 96x576x1x1]
  %MobilenetV3small/expanded_conv_9/